# TopoQuery: Explainable Query Recommendations via Persistent Homology

**Interactive Demo Notebook**

This notebook demonstrates how TopoQuery uses topological data analysis (TDA) and persistent homology to generate explainable SQL query recommendations.

---

## 📋 Overview

TopoQuery consists of 5 main steps:
1. **Load SQL queries** from a workload
2. **Extract features** (structural, semantic, contextual)
3. **Compute persistent homology** across multiple distance scales  
4. **Visualize topology** with persistence diagrams, barcodes, and projections
5. **Generate recommendations** based on topological proximity

We'll use the **Palmer Penguins dataset** with 31 diverse SQL queries.

## 🔧 Setup & Installation

First, let's import all necessary libraries:

In [ ]:
import sys
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import TopoQuery modules
from create_database import create_penguin_database
from generate_queries import generate_penguin_queries
from parse_queries import QueryParser
from topological_analysis import TopologicalAnalyzer
from recommend_queries import QueryRecommender
from visualize_results import QueryVisualizer

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ All libraries imported successfully!")

## 📊 Step 1: Load Dataset & Generate Queries

We'll create a SQLite database with the Palmer Penguins dataset and generate a diverse workload of SQL queries.

In [ ]:
# Create database
db_path = "data/penguins.db"
os.makedirs("data", exist_ok=True)

print("Creating Palmer Penguins database...")
create_penguin_database(db_path)

# Verify database
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM penguins LIMIT 5", conn)
print(f"\n✅ Database created with {pd.read_sql_query('SELECT COUNT(*) as count FROM penguins', conn)['count'][0]} penguin observations")
print("\nFirst 5 rows:")
display(df)
conn.close()

In [ ]:
# Generate SQL queries
queries_dir = "queries"
os.makedirs(queries_dir, exist_ok=True)

print("Generating diverse SQL query workload...")
queries = generate_penguin_queries(db_path, queries_dir)

print(f"\n✅ Generated {len(queries)} SQL queries")
print("\nExample queries:")
for i, q in enumerate(queries[:3], 1):
    print(f"\n{i}. {q}")

## 🔍 Step 2: Extract Query Features

Extract structural, semantic, and contextual features from each query.

In [ ]:
# Parse queries and extract features
print("Parsing queries and extracting features...")
parser = QueryParser()
features_list = []

for query in queries:
    features = parser.parse_query(query)
    features_list.append(features)

# Convert to DataFrame
features_df = pd.DataFrame(features_list)

print(f"\n✅ Extracted {len(features_df.columns)} features from {len(queries)} queries")
print(f"\nFeature dimensions: {features_df.shape}")
print("\nFirst few features:")
display(features_df.head())

## 🔬 Step 3: Compute Persistent Homology

Apply persistent homology to identify stable topological features.

In [ ]:
# Initialize topological analyzer
print("Computing persistent homology...")
analyzer = TopologicalAnalyzer()

# Compute with Euclidean metric
result = analyzer.analyze(
    features_df.values,
    distance_metric='euclidean',
    max_dimension=1
)

# Analyze persistence
persistence_dim0 = result['persistence'][0]
lifetimes = [(death - birth) for birth, death in persistence_dim0 if death != float('inf')]

print(f"\n✅ Found {len(persistence_dim0)} topological features")
print(f"\nTop 5 longest-lived features:")
for i, lifetime in enumerate(sorted(lifetimes, reverse=True)[:5], 1):
    print(f"  {i}. Lifetime: {lifetime:.4f}")

## 📈 Step 4: Visualize Topological Structure

Generate persistence diagrams, barcodes, and projections.

In [ ]:
# Create visualizer
os.makedirs("outputs/visualizations", exist_ok=True)
visualizer = QueryVisualizer()

print("Generating visualizations...\n")

# Persistence Diagram
print("1. Creating persistence diagram...")
visualizer.plot_persistence_diagram(
    result['persistence'],
    title="Persistence Diagram (Euclidean)",
    save_path="outputs/visualizations/demo_persistence_diagram.png"
)
plt.show()

# Persistence Barcode
print("2. Creating persistence barcode...")
visualizer.plot_persistence_barcode(
    result['persistence'],
    title="Persistence Barcode (Euclidean)",
    save_path="outputs/visualizations/demo_persistence_barcode.png"
)
plt.show()

print("\n✅ Visualizations complete!")

In [ ]:
# PCA Projection
from sklearn.decomposition import PCA

print("3. Creating PCA projection...")
pca = PCA(n_components=2)
pca_coords = pca.fit_transform(features_df.values)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_coords[:, 0], pca_coords[:, 1], 
                     c=range(len(pca_coords)), cmap='tab10', 
                     s=100, alpha=0.7, edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='Query ID')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
plt.title('PCA Projection of Query Feature Space', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/visualizations/demo_pca_projection.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nPCA preserves {sum(pca.explained_variance_ratio_):.1%} of total variance")

## 🎯 Step 5: Generate Query Recommendations

Use topological proximity to recommend similar queries with explanations.

In [ ]:
# Initialize recommender
recommender = QueryRecommender(
    queries=queries,
    features=features_df.values,
    persistence_data=result['persistence']
)

# Select a target query
target_idx = 10  # Change this to explore different queries
target_query = queries[target_idx]

print(f"🎯 Target Query (ID: {target_idx}):")
print(f"\n{target_query}\n")
print("=" * 80)

# Get recommendations
recommendations = recommender.recommend(target_idx, top_k=5)

print(f"\n📋 Top 5 Topologically Similar Queries:\n")

for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. Query ID: {rec['query_id']} | Score: {rec['score']:.4f}")
    print(f"   {rec['query']}")
    print(f"   📊 Explanation: {rec['explanation']}")
    print("   " + "-" * 76)

In [ ]:
# Visualize recommendations in PCA space
plt.figure(figsize=(12, 8))

# Plot all queries
plt.scatter(pca_coords[:, 0], pca_coords[:, 1],
           c='lightgray', s=80, alpha=0.5, label='Other queries')

# Highlight recommendations
rec_indices = [rec['query_id'] for rec in recommendations]
rec_coords = pca_coords[rec_indices]
plt.scatter(rec_coords[:, 0], rec_coords[:, 1],
           c='blue', s=150, alpha=0.7, edgecolors='black',
           linewidth=1.5, label='Recommendations', marker='o')

# Highlight target
target_coord = pca_coords[target_idx]
plt.scatter(target_coord[0], target_coord[1],
           c='red', s=300, alpha=1.0, edgecolors='black',
           linewidth=2, label='Target Query', marker='*')

# Draw connections
for rec_coord in rec_coords:
    plt.plot([target_coord[0], rec_coord[0]], 
            [target_coord[1], rec_coord[1]],
            'k--', alpha=0.3, linewidth=0.8)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
plt.title('TopoQuery Recommendations in PCA Space', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/visualizations/demo_recommendations.png', dpi=300, bbox_inches='tight')
plt.show()

## 📊 Summary

✅ **TopoQuery Demo Complete!**

### Key Takeaways:
1. **Persistent Homology** identifies stable query clusters across distance scales
2. **Topological Features** reveal structural patterns syntax-based methods miss
3. **Visual Explanations** make abstract concepts accessible
4. **Explainable Recommendations** help users understand *why* queries are related

### Next Steps:
- Try different target queries by changing `target_idx`
- Explore different distance metrics ('euclidean', 'cosine', 'mahalanobis')
- Generate your own SQL workload and analyze it!

---

**Paper:** TopoQuery: Explainable Query Recommendations via Persistent Homology  
**Author:** Sudhanva Athreya | University of Utah  
**Course:** CS 6360 - Advanced Database Systems